In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator

img_width, img_height = 224, 224


from keras.applications import ResNet152V2
from keras.models import Sequential
from keras.layers import Dense, Flatten

resnet = ResNet152V2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

for layer in resnet.layers:
    layer.trainable = False

model = Sequential()
model.add(resnet)
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

234545216/234545216 [==============================] - 6s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152v2 (Functional)    (None, 7, 7, 2048)        58331648  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 

In [ ]:
epochs = 15

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/new Dataset Senior - Kopya/TRAIN',
    target_size=(img_width, img_height),
    batch_size=64,
    class_mode='categorical')


test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/new Dataset Senior - Kopya/TEST',
    target_size=(img_width, img_height),
    batch_size=64,
    class_mode='categorical',
    shuffle=False)


Found 11700 images belonging to 4 classes.
Found 3002 images belonging to 4 classes.


In [ ]:
train_generator.class_indices

{'COVID': 0, 'LUNG OPACITY': 1, 'NORMAL': 2, 'PNEUMONIA': 3}

In [ ]:
model.fit(
    train_generator,
    epochs=epochs
   )

model.save('pnomoni_model_Senior_sunum_real.h5')

Epoch 1/15
183/183 [==============================] - 4594s 25s/step - loss: 0.5315 - accuracy: 0.7931
Epoch 2/15
183/183 [==============================] - 63s 345ms/step - loss: 0.3472 - accuracy: 0.8680
Epoch 3/15
183/183 [==============================] - 63s 341ms/step - loss: 0.2813 - accuracy: 0.8920
Epoch 4/15
183/183 [==============================] - 61s 330ms/step - loss: 0.2406 - accuracy: 0.9104
Epoch 5/15
183/183 [==============================] - 61s 331ms/step - loss: 0.2196 - accuracy: 0.9196
Epoch 6/15
183/183 [==============================] - 59s 324ms/step - loss: 0.1803 - accuracy: 0.9345
Epoch 7/15
183/183 [==============================] - 60s 329ms/step - loss: 0.1570 - accuracy: 0.9427
Epoch 8/15
183/183 [==============================] - 60s 325ms/step - loss: 0.1448 - accuracy: 0.9474
Epoch 9/15
183/183 [==============================] - 60s 330ms/step - loss: 0.1217 - accuracy: 0.9560
Epoch 10/15
183/183 [==============================] - 60s 327ms/step - l

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.save('/content/drive/MyDrive/pnomoni_model_Senior_sunum_real.h5')

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

predictions = model.predict(test_generator)
true_labels = test_generator.classes

from sklearn.metrics import classification_report
print(classification_report(true_labels, predictions.argmax(axis=1)))

from sklearn.metrics import confusion_matrix, precision_score, recall_score
conf_matrix = confusion_matrix(true_labels, predictions.argmax(axis=1))
print("Confusion Matrix:")
print(conf_matrix)


precision = precision_score(true_labels, predictions.argmax(axis=1), average=None)
recall = recall_score(true_labels, predictions.argmax(axis=1), average=None)
print("Precision:", precision)
print("Recall:", recall)


47/47 [==============================] - 1055s 23s/step - loss: 0.2332 - accuracy: 0.9127
Test Loss: 0.23324528336524963
Test Accuracy: 0.9127248525619507
47/47 [==============================] - 18s 336ms/step
              precision    recall  f1-score   support

           0       0.85      0.97      0.91       875
           1       0.95      0.86      0.90       768
           2       0.91      0.84      0.88       822
           3       0.97      1.00      0.99       537

    accuracy                           0.91      3002
   macro avg       0.92      0.92      0.92      3002
weighted avg       0.92      0.91      0.91      3002

Confusion Matrix:
[[848   1  23   3]
 [ 59 661  46   2]
 [ 86  33 694   9]
 [  0   0   0 537]]
Precision: [0.85397784 0.95107914 0.9095675  0.97459165]
Recall: [0.96914286 0.86067708 0.84428224 1.        ]


In [ ]:
from keras.preprocessing import image
import numpy as np

img_path = '/content/drive/MyDrive/new Dataset Senior - Kopya/TEST2/LUNG OPACITY/10289.png'
img = image.load_img(img_path, target_size=(img_width, img_height))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

predictions = model.predict(img_array)

# Get the predicted class label
predicted_class = np.argmax(predictions)
class_names = ['Covid', 'Lung Opacity', 'Normal', 'PNEUMONIA']
print("Prediction:", class_names[predicted_class])

1/1 [==============================] - 0s 38ms/step
Prediction: Lung Opacity
